Annika Chan
word2vec notebook
cleaned python file can be found without notes in word2vec.py

In [10]:
# downloading the packages for the environment

#!pip install gensim
#!pip install matplotlib
#!pip install scikit-learn==1.3.1

In [11]:
import gensim
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

creating/implementing the pretrained model
this will be used later to determine biases in language
data can be found here - https://www.kaggle.com/datasets/adarshsng/googlenewsvectors

In [12]:
model_path = "/Users/anni/Library/CloudStorage/OneDrive-EastsidePreparatorySchool/High School/12th Grade/IP/GoogleNews-vectors-negative300.bin"
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

## Part 1: exploring word2vec

In Word2Vec, each word is represented as a vector in a multi-dimensional space with each index in the vector representing a specific feature and that index value being the mathematical representation of the word described by that feature dimension(e.g. [0.3, 0.8] could be a vector for "grandpa" and the second value represents the age dimension)

Word2Vec has two different architectures: skipgram and continous bag of words (cbow)
* skip gram: based on a target word, the model will predict the words around it
* cbow: based on the words in the sentence around the target word, the model will predict the missing word 


In [13]:
# Finding similar words
word = 'loud'
similar_words = model.most_similar(word, topn=10)
print(f"Words similar to {word}:")

for word, score in similar_words:
    print(f"\t{word} - {score}\n")


Words similar to loud:
	noisy - 0.6621273159980774

	loudly - 0.636400580406189

	fer_cryin - 0.6316129565238953

	deafening - 0.6294395327568054

	Loud - 0.6140207052230835

	loudest - 0.5992903113365173

	obnoxiously_loud - 0.5985252261161804

	ear_splitting - 0.597548246383667

	screeching_noises - 0.5830792188644409

	muffled - 0.5812103152275085

